In [1413]:
from os import defpath
from random import choice, randint
from sys import argv
from time import time
import time

In [1414]:
# Permutations for each move
MOVES_LIST = {
    "U":  [2,   0,   3,   1,  20,  21,   6,   7,   4,   5,  10,  11,  12,  13,  14,  15,   8,   9,  18,  19,  16,  17,  22,  23],
    "U'": [1,   3,   0,   2,   8,   9,   6,   7,  16,  17,  10,  11,  12,  13,  14,  15,  20,  21,  18,  19,   4,   5,  22,  23],
    "R":  [0,   9,   2,  11,   6,   4,   7,   5,   8,  13,  10,  15,  12,  22,  14,  20,  16,  17,  18,  19,   3,  21,   1,  23],
    "R'": [0,  22,   2,  20,   5,   7,   4,   6,   8,   1,  10,   3,  12,   9,  14,  11,  16,  17,  18,  19,  15,  21,  13,  23],
    "F":  [0,   1,  19,  17,   2,   5,   3,   7,  10,   8,  11,   9,   6,   4,  14,  15,  16,  12,  18,  13,  20,  21,  22,  23],
    "F'": [0,   1,   4,   6,  13,   5,  12,   7,   9,  11,   8,  10,  17,  19,  14,  15,  16,   3,  18,   2,  20,  21,  22,  23],
    "D":  [0,   1,   2,   3,   4,   5,  10,  11,   8,   9,  18,  19,  14,  12,  15,  13,  16,  17,  22,  23,  20,  21,   6,   7],
    "D'": [0,   1,   2,   3,   4,   5,  22,  23,   8,   9,   6,   7,  13,  15,  12,  14,  16,  17,  10,  11,  20,  21,  18,  19],
    "L":  [23,  1,  21,   3,   4,   5,   6,   7,   0,   9,   2,  11,   8,  13,  10,  15,  18,  16,  19,  17,  20,  14,  22,  12],
    "L'": [8,   1,  10,   3,   4,   5,   6,   7,  12,   9,  14,  11,  23,  13,  21,  15,  17,  19,  16,  18,  20,   2,  22,   0],
    "B":  [5,   7,   2,   3,   4 , 15,   6,  14,   8,   9,  10,  11,  12,  13,  16,  18,   1,  17,   0,  19,  22,  20,  23,  21],
    "B'": [18, 16,   2,   3,   4,   0,   6,   1,   8,   9,  10,  11,  12,  13,   7,   5,  14,  17,  15,  19,  21,  23,  20,  22],
}

# Potential Values for distance in Heuristics Function and PDDL
# Sticker Indices for each corner pieces
CORNER_LIST = [(10, 12, 19), ( 6, 11, 13),
               ( 2,  8, 17), ( 3,  4,  9),
               (14, 18, 23), ( 7, 15, 22),
               ( 0, 16, 21), ( 1,  5, 20)]

# 3D Coordinates for each corner
COORDINATES_LIST = [(0, 0, 0), (0, 1, 0), (1, 0, 0), (1, 1, 0),
                    (0, 0, 1), (0, 1, 1), (1, 0, 1), (1, 1, 1)]

In [1415]:
class Cube2x2:

    # Function: Initialize Cube
    def __init__(self, cube = "WWWW RRRR GGGG YYYY OOOO BBBB", movesList=[], depth=0):
        self.movesList = movesList
        self.depth = depth
        self.state = self.checkCube(cube)

    # Function: Check Cube String
    def checkCube(self, state):
        state = state.replace(" ", "")
        state = state.upper()
        if len(state) != 24:
            raise ValueError("State must have only 24 Stickers.")
        colors = ["W", "R", "G", "Y", "O", "B"]
        for color in colors:
            if state.count(color) != 4:
                raise ValueError("State must exactly have 4 Stickers of each Color.")
        return state

    # Function: Print Cube
    def print(self):
        print("    {}{}              \n".format(self.state[0], self.state[1]))
        print("    {}{}              \n".format(self.state[2], self.state[3]))
        print(" {}{} {}{} {}{} {}{}  \n".format(self.state[16], self.state[17], self.state[8], self.state[9], self.state[4], self.state[5], self.state[20], self.state[21]))
        print(" {}{} {}{} {}{} {}{}  \n".format(self.state[18], self.state[19], self.state[10], self.state[11], self.state[6], self.state[7], self.state[22], self.state[23]))
        print("    {}{}              \n".format(self.state[12], self.state[13]))
        print("    {}{}              \n".format(self.state[14], self.state[15]))

    # Function: Return Current State
    def returnCurrentState(self):
        return self.state

    # Function: Check if Goal State Reached
    def checkIfGoalState(self):
        for i in range(0, len(self.state), 4):
            side = self.state[i:i+4]
            if side.count(side[0]) != 4:
                return False
        return True

    # Function: Execute a single given Move to the State
    def executeMove(self, move):
        # For Invalid Move
        if move not in MOVES_LIST.keys():
            raise ValueError(f"Invalid Move: {move}")

        # Sticker Rotation for a Given Move
        permutations = MOVES_LIST[move]
        self.state = "".join([self.state[i] for i in permutations])
        return self.state

    # Function: Execute a set of given Moves to the State
    def executeMoves(self, moves):
        for move in moves.split():
            print(move)
            self.executeMove(move)

    # Function: Shuffle Cube with N Moves
    def shuffleCube(self, N):
        # Choose n random moves from the list of moves and creates an algorithm to apply
        moves = list(MOVES_LIST.keys())
        shuffle = " ".join([choice(moves) for i in range(N)])
        self.executeMoves(shuffle)


In [1416]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

def create_tree(depth, branching_factor):
    if depth == 0:
        return None

    root = Node(f"Node at depth {depth}")
    for i in range(branching_factor):
        child = create_tree(depth - 1, branching_factor)
        if child:
            root.children.append(child)

    return root

def dfs(node):
    if node is not None:
        print(node.value, end=' ')
        for child in node.children:
            dfs(child)

In [1417]:

# def dfs(graph, start, visited=None):
#     if visited is None:
#         visited = set()

#     visited.add(start)
#     print(start, end=' ')

#     for neighbor in graph[start] - visited:
#         dfs(graph, neighbor, visited)
    

def shawnDfs(cubeState, movesList, solution, currentLayer):
    if currentLayer == 0:
        for move in movesList:
            print(move)
            solution.append(move)
        return solution
    # solution.append(move)

    
    x = shawnDfs(cubeState, movesList, solution, currentLayer-1)
    return x

def InverseMove(move):
    if move.endswith("'"):
        return move[:-1]
    elif move.endswith("2"):
        return move
    else:
        return move + "'"    

def shawnSolveRubiksCube(initialCubeState, movesList):
    depthLimit = 0
    currentLayer = 3
    init = initialCubeState
    cube = Cube2x2(initialCubeState, movesList)
    solution = []
    maxCombinations = len(movesList)
    # movesList = {
    #     'U': [2, 0, 3, 1, 20, 21, 6, 7, 4, 5, 10, 11, 12, 13, 14, 15, 8, 9, 18, 19, 16, 17, 22, 23],
    #     'D': [1, 3, 0, 2, 8, 9, 6, 7, 16, 17, 10, 11, 12, 13, 14, 15, 20, 21, 18, 19, 4, 5, 22, 23],
    #     'R': [0, 9, 2, 11, 6, 4, 7, 5, 8, 13, 10, 15, 12, 22, 14, 20, 16, 17, 18, 19, 3, 21, 1, 23],
    #     'L': [0, 9, 2, 11, 6, 4, 7, 5, 8, 13, 10, 15, 12, 22, 14, 20, 16, 17, 18, 19, 3, 21, 1, 23]
    # }
    print(cube.returnCurrentState())
    # shawnDfs(cube.returnCurrentState(), cube.movesList, solution, currentLayer)
    # for move in cube.movesList:
    for i in movesList:
        # print(i)
        cube.executeMove(i)
        solution.append(i)
        if cube.checkIfGoalState():
            print(cube.returnCurrentState())
            break
        # for i in range(2):
        for j in movesList:
            # print(move)
            cube.executeMove(j)
            if cube.checkIfGoalState():
                print(cube.returnCurrentState())
                solution.append(j)
                break

            inverseMove = InverseMove(j)
            cube.executeMove(j)
        
        if cube.checkIfGoalState():
            break
        else:
            solution = []
    #         # print(cube.returnCurrentState())
    print("pop")
    return solution

def solveRubiksCube(initialCubeState, movesList):
    depthLimit = 0
    solution = []

    cube = Cube2x2(initialCubeState, movesList)
    # cube = Cube2x2()

    while not DFS(cube, depthLimit, solution):
        depthLimit += 1

    cube.print()
    return solution


In [1418]:
cube = Cube2x2()
cube.returnCurrentState()

'WWWWRRRRGGGGYYYYOOOOBBBB'

In [1419]:
cube = Cube2x2()
cube.executeMove("D")
cube.returnCurrentState()

'WWWWRRGGGGOOYYYYOOBBBBRR'

# TEST

In [1420]:
# 1 trial run
cube = Cube2x2()
cube.shuffleCube(3)
# cube.print()
# solveRubiksCube(cube.returnCurrentState(), MOVES_LIST)
# graph = {
#     'A': {'B', 'C'},
#     'B': {'A', 'D', 'E'},
#     'C': {'A', 'F', 'G'},
#     'D': {'B'},
#     'E': {'B'},
#     'F': {'C'},
#     'G': {'C'}
# }

# dfs(graph, 'A')


# tree_depth = 3
# branching_factor = 3

# # Create the tree
# root_node = create_tree(tree_depth, branching_factor)

# # Perform DFS on the tree starting from the root
# dfs(root_node)
# YYWWROROGGGGYYWWROROBBBB
# solution = shawnSolveRubiksCube('YYWWROROGGGGYYWWROROBBBB', MOVES_LIST)
solution = shawnSolveRubiksCube(cube.returnCurrentState(), MOVES_LIST)
cube.returnCurrentState()
print(solution)

U'
R
U'
OGWWOYRGBBGYYBYRWROORGWB
pop
[]
